In [1]:
import cv2
import os
import json
import numpy as np

images_dir = './samples'
input = 'input.json'
output = 'output.json'
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 0
# Load the image
original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))

# Resize the image
original_image = cv2.resize(original_image, (0, 0), fx=0.15, fy=0.15)
image = original_image.copy()

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply median blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)
# Harris Corner Detection
harris_image = image.copy()
harris_num_legos = 0
# list of contours
dst = cv2.cornerHarris(blur, 2, 3, 0.04)
dst = cv2.dilate(dst, None)
threshold = 0.01 * dst.max()
corner_image = image.copy()
corner_image[dst > threshold] = [0,0,255]
ret, dst_thresh = cv2.threshold(dst, threshold, 255, 0)
dst_thresh = np.uint8(dst_thresh)
contours, _ = cv2.findContours(dst_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 50:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(harris_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        harris_num_legos += 1


canny_image = image.copy()
cannys_found_contours = [] 

edges = cv2.Canny(blur, 30, 50)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)
    
    if area > 10:
        to_place = True
        cannys_found_contours_copy = cannys_found_contours.copy()
        for existing_contour_info in cannys_found_contours:
            existing_contour = existing_contour_info['contour']
            existing_x, existing_y, existing_w, existing_h = existing_contour_info['x'], existing_contour_info['y'], existing_contour_info['w'], existing_contour_info['h']
            
            intersect_x1 = max(x, existing_x)
            intersect_y1 = max(y, existing_y)
            intersect_x2 = min(x + w, existing_x + existing_w)
            intersect_y2 = min(y + h, existing_y + existing_h)
            intersect_area = max(0, intersect_x2 - intersect_x1) * max(0, intersect_y2 - intersect_y1)


            # if the area of intersection is greater the half of the size of the larger contour, then we consider it as the same contour

            if intersect_area > 0:
                if((w*h) > (existing_w * existing_h)):
                    cannys_found_contours_copy.remove(existing_contour_info)    
                else:
                    to_place = False
                    break
                

        if to_place:
            contour_info = {
                'x': x,
                'y': y,
                'w': w,
                'h': h,
                'contour': contour 
            }

            cannys_found_contours_copy.append(contour_info)
            cannys_found_contours = cannys_found_contours_copy

for found_contour in cannys_found_contours:
    x, y, w, h = found_contour['x'], found_contour['y'], found_contour['w'], found_contour['h']
    cv2.rectangle(canny_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    


cv2.imshow('Original Image', original_image)
cv2.imshow('Corner Detection', corner_image) 
cv2.imshow('Blur', blur)
cv2.imshow('Harris Corner Detection', harris_image)
cv2.imshow('Canny Edge Detection', canny_image)
cv2.imshow('Canny Corner Detection', edges)


cv2.waitKey(0)
cv2.destroyAllWindows()
